In [1]:
import cv2

record = False

cap = cv2.VideoCapture(0)

if (cap.isOpened() == False): 
  print("Unable to read camera feed")

frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

out = cv2.VideoWriter('output.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 10, (frame_width,frame_height))

while(True):
  ret, frame = cap.read()
  k = cv2.waitKey(1)

  if ret == True: 
    cv2.imshow('frame',frame)

    # press space key to start recording
    if k%256 == 32:
        record = True

    if record:
        out.write(frame) 

    # press q key to close the program
    if k & 0xFF == ord('q'):
        break

  else:
     break  

cap.release()
out.release()

cv2.destroyAllWindows()

In [3]:
%matplotlib inline

In [4]:
import numpy as np

In [5]:
# Load the networks inputs

# Useful Constants
# Output classes to learn how to classify
LABELS = [    
    "JUMPING",
    "JUMPING_JACKS",
    "BOXING",
    "WAVING_2HANDS",
    "WAVING_1HAND",
    "CLAPPING_HANDS"

] 
n_steps = 32 # 32 timesteps per series

def load_X(X_path):
    file = open(X_path, 'r')
    X_ = np.array(
        [elem for elem in [
            row.split(',') for row in file
        ]], 
        dtype=np.float32
    )
    file.close()
    blocks = int(len(X_) / n_steps)
    
    X_ = np.array(np.split(X_,blocks))

    return X_ 

# Load the networks outputs
def load_y(y_path):
    file = open(y_path, 'r')
    y_ = np.array(
        [elem for elem in [
            row.replace('  ', ' ').strip().split(' ') for row in file
        ]], 
        dtype=np.int32
    )
    file.close()
    
    # for 0-based indexing 
    return y_ - 1

In [6]:
import torch

In [7]:
import torch.nn as nn

In [8]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [9]:
import torch
import torch.nn as nn
import numpy as np
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
class LSTM(nn.Module):
    
    def __init__(self,input_dim,hidden_dim,output_dim,layer_num):
        super(LSTM,self).__init__()
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.lstm = torch.nn.LSTM(input_dim,hidden_dim,layer_num,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim,output_dim)
        self.bn = nn.BatchNorm1d(32)
        
    def forward(self,inputs):
        x = self.bn(inputs)
        lstm_out,(hn,cn) = self.lstm(x)
        out = self.fc(lstm_out[:,-1,:])
        return out


In [10]:
only_keypoint1 = []
only_keypoint = []
arr1 = np.array([[307.589,162.976,319.364,205.944,293.267,204.68,285.434,250.281,277.616,286.729,349.276,208.539,357.095,255.523,349.362,297.17,297.183,290.662,307.599,351.938,308.905,401.454,329.758,291.958,333.701,357.132,337.546,411.953,304.994,160.291,315.459,160.269,0,0,329.752,161.651]])  
only_keypoint = np.array(arr1,  dtype=np.float)
arr2 = np.array([307.589,162.976,319.364,205.944,293.267,204.68,285.434,250.281,277.616,286.729,349.276,208.539,357.095,255.523,349.362,297.17,297.183,290.662,307.599,351.938,308.905,401.454,329.758,291.958,333.701,357.132,337.546,411.953,304.994,160.291,315.459,160.269,0,0,329.752,161.651])  
only_keypoint=np.vstack(([only_keypoint, arr2]))
only_keypoint1=np.array([only_keypoint],dtype=np.float)
#only_keypoint1=np.array([only_keypoint],dtype=np.float)
print(only_keypoint1)
print(only_keypoint1.shape)
#a = only_keypoint.reshape (1,32, 36)

[[[307.589 162.976 319.364 205.944 293.267 204.68  285.434 250.281
   277.616 286.729 349.276 208.539 357.095 255.523 349.362 297.17
   297.183 290.662 307.599 351.938 308.905 401.454 329.758 291.958
   333.701 357.132 337.546 411.953 304.994 160.291 315.459 160.269
     0.      0.    329.752 161.651]
  [307.589 162.976 319.364 205.944 293.267 204.68  285.434 250.281
   277.616 286.729 349.276 208.539 357.095 255.523 349.362 297.17
   297.183 290.662 307.599 351.938 308.905 401.454 329.758 291.958
   333.701 357.132 337.546 411.953 304.994 160.291 315.459 160.269
     0.      0.    329.752 161.651]]]
(1, 2, 36)


In [11]:

inputs = load_X("demofile3.txt")
print(inputs.shape)
inputs=torch.from_numpy(inputs)
inputs



(1, 32, 36)


tensor([[[307.5890, 162.9760, 319.3640,  ...,   0.0000, 329.7520, 161.6510],
         [307.5670, 162.9790, 319.3620,  ...,   0.0000, 328.5270, 161.6550],
         [306.2980, 162.9510, 319.3510,  ...,   0.0000, 328.4950, 161.6810],
         ...,
         [293.2910, 122.5340, 307.6760,  ..., 128.9530, 315.4380, 119.8840],
         [289.3920, 140.7430, 307.6150,  ...,   0.0000, 315.3930, 139.4080],
         [295.8480, 161.6580, 307.6280,  ..., 160.3310, 314.1120, 160.2640]]])

In [12]:
n_hidden = 128
n_joints = 18*2
n_categories = 6
n_layer = 3
#model = torch.load('lstm_6_bn.pkl')

In [13]:
model = LSTM(n_joints,n_hidden,n_categories,n_layer)
model.load_state_dict(torch.load('lstm_6_bn.pkl'))
model.eval()

LSTM(
  (lstm): LSTM(36, 128, num_layers=3, batch_first=True)
  (fc): Linear(in_features=128, out_features=6, bias=True)
  (bn): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [14]:
#category_tensor, inputs = randomTrainingExampleBatch(1,'test',0)
#category = LABELS[int(category_tensor[0])]
inputs = inputs.to(device)
model.cuda()
output = model(inputs)
top_n, top_i = output.topk(1)
category_i = top_i[0].item()
category = LABELS[category_i]
category_ii = LABELS.index(category)
category ,category_ii,inputs

('BOXING',
 2,
 tensor([[[307.5890, 162.9760, 319.3640,  ...,   0.0000, 329.7520, 161.6510],
          [307.5670, 162.9790, 319.3620,  ...,   0.0000, 328.5270, 161.6550],
          [306.2980, 162.9510, 319.3510,  ...,   0.0000, 328.4950, 161.6810],
          ...,
          [293.2910, 122.5340, 307.6760,  ..., 128.9530, 315.4380, 119.8840],
          [289.3920, 140.7430, 307.6150,  ...,   0.0000, 315.3930, 139.4080],
          [295.8480, 161.6580, 307.6280,  ..., 160.3310, 314.1120, 160.2640]]],
        device='cuda:0'))

# OpenPose

In [15]:
from argparse import ArgumentParser
import json
import os

import cv2
import numpy as np
from modules.input_reader import VideoReader, ImageReader
from modules.draw import Plotter3d, draw_poses
from modules.parse_poses import parse_poses,get_root_relative_poses
from modules.inference_engine_pytorch import InferenceEnginePyTorch
net = InferenceEnginePyTorch('human-pose-estimation-3d.pth', 'GPU',)

har_addframe = []
har_32frame = []
def rotate_poses(poses_3d, R, t):
    R_inv = np.linalg.inv(R)
    for pose_id in range(len(poses_3d)):
        pose_3d = poses_3d[pose_id].reshape((-1, 4)).transpose()
        pose_3d[0:3, :] = np.dot(R_inv, pose_3d[0:3, :] - t)
        poses_3d[pose_id] = pose_3d.transpose().reshape(-1)

    return poses_3d
frame_provider = VideoReader(0)
is_video = True
base_height = 256 #args.height_size
fx = -1 #args.fx

delay = 1
esc_code = 27
p_code = 112
space_code = 32
mean_time = 0
stride = 8
demframe = 0

category = ''
category_ii = ''
for frame in frame_provider:
    current_time = cv2.getTickCount()
    if frame is None:
        break
    input_scale = base_height / frame.shape[0]
    scaled_img = cv2.resize(frame, dsize=None, fx=input_scale, fy=input_scale)
    scaled_img = scaled_img[:, 0:scaled_img.shape[1] - (scaled_img.shape[1] % stride)]  # better to pad, but cut out for demo
    if fx < 0:  # Focal length is unknown
        fx = np.float32(0.8 * frame.shape[1])
    inference_result = net.infer(scaled_img)
    poses_3d, poses_2d, features_shape,keypoints = get_root_relative_poses(inference_result)
    
    
    poses_3d1, poses_2d1 = parse_poses(inference_result, input_scale, stride, fx, is_video)
    for pose_id in range(len(poses_2d)):
        #print(poses_2d[pose_id])
        #print(keypoints[pose_id])
        print("dem frame",demframe)
        if(demframe==0):
            har_addframe = np.array([keypoints[0]])
            har_addframe[har_addframe<0] = 0
            print("Dautien",har_addframe)
            print("SHAPE",har_addframe.shape)
        if(demframe>0 and demframe<32):   
            har_addframe = np.vstack(([har_addframe, [keypoints[0]]]))
            har_addframe[har_addframe<0] = 0
            print("SHAPE",har_addframe.shape)
        if(demframe==31):
            har_32frame=np.array([har_addframe])
            print("SHAPEPRO", har_32frame.shape)
            print("SHAPEPRO", har_32frame)
            inputs=torch.from_numpy(har_32frame)
            print("input",inputs)
            inputs = inputs.to(device)
            model.cuda()
            output = model(inputs)
            top_n, top_i = output.topk(1)
            category_i = top_i[0].item()
            category = LABELS[category_i]
            category_ii = LABELS.index(category)
            print("Tong hop)", category ,category_ii)
        
        demframe+=1
        if(demframe >= 32):
            har_addframe =[]
            demframe = 0
            har_32frame = []
        #pose = np.array(poses_2d[pose_id][0:-1]).reshape((-1, 3)).transpose()
        #print(poses_2d[pose_id])
        #np.append(arr, np.array(pose), axis=0)
        #print(len(poses_2d[0]))
    #break
    edges = []
    
    
    draw_poses(frame, poses_2d1)
    current_time = (cv2.getTickCount() - current_time) / cv2.getTickFrequency()
    if mean_time == 0:
        mean_time = current_time
    else:
        mean_time = mean_time * 0.95 + current_time * 0.05
    cv2.putText(frame, 'FPS: {}'.format(int(1 / mean_time * 10) / 10),
                (40, 80), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 255))
    cv2.putText(frame, category ,
                (40, 120), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0))
    cv2.imshow('ICV 3D Human Pose Estimation', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

#### Cannot load fast pose extraction, switched to legacy slow implementation. ####
dem frame 0
Dautien [[22.5  17.5  25.25 12.5  32.25 17.25 36.5  26.5  34.5  30.5   0.    0.
   0.    0.    0.    0.   11.5  18.5   6.25 31.5   0.    0.    0.    0.
   0.    0.    0.    0.   21.5   9.5  27.5   9.25 17.5   7.5  30.5   7.25]]
SHAPE (1, 36)
dem frame 1
SHAPE (2, 36)
dem frame 2
SHAPE (3, 36)
dem frame 3
SHAPE (4, 36)
dem frame 4
SHAPE (5, 36)
dem frame 5
SHAPE (6, 36)
dem frame 6
SHAPE (7, 36)
dem frame 7
SHAPE (8, 36)
dem frame 8
SHAPE (9, 36)
dem frame 9
SHAPE (10, 36)
dem frame 10
SHAPE (11, 36)
dem frame 11
SHAPE (12, 36)
dem frame 12
SHAPE (13, 36)
dem frame 13
SHAPE (14, 36)
dem frame 14
SHAPE (15, 36)
dem frame 15
SHAPE (16, 36)
dem frame 16
SHAPE (17, 36)
dem frame 17
SHAPE (18, 36)
dem frame 18
SHAPE (19, 36)
dem frame 19
SHAPE (20, 36)
dem frame 20
SHAPE (21, 36)
dem frame 21
SHAPE (22, 36)
dem frame 22
SHAPE (23, 36)
dem frame 23
SHAPE (24, 36)
dem frame 24
SHAPE (25, 36)
dem fra

dem frame 14
SHAPE (15, 36)
dem frame 15
SHAPE (16, 36)
dem frame 16
SHAPE (17, 36)
dem frame 17
SHAPE (18, 36)
dem frame 18
SHAPE (19, 36)
dem frame 19
SHAPE (20, 36)
dem frame 20
SHAPE (21, 36)
dem frame 21
SHAPE (22, 36)
dem frame 22
SHAPE (23, 36)
dem frame 23
SHAPE (24, 36)
dem frame 24
SHAPE (25, 36)
dem frame 25
SHAPE (26, 36)
dem frame 26
SHAPE (27, 36)
dem frame 27
SHAPE (28, 36)
dem frame 28
SHAPE (29, 36)
dem frame 29
SHAPE (30, 36)
dem frame 30
SHAPE (31, 36)
dem frame 31
SHAPE (32, 36)
SHAPEPRO (1, 32, 36)
SHAPEPRO [[[22.25  2.25  0.   ...  0.    0.    0.  ]
  [22.25  2.25  0.   ...  0.    0.    0.  ]
  [22.25  1.5   0.   ...  0.    0.    0.  ]
  ...
  [22.5   2.5   0.   ...  0.    0.    0.  ]
  [22.5   2.5   0.   ...  0.    0.    0.  ]
  [22.5   2.5   0.   ...  0.    0.    0.  ]]]
input tensor([[[22.2500,  2.2500,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [22.2500,  2.2500,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [22.2500,  1.5000,  0.0000,  ...,  0.

Tong hop) WAVING_2HANDS 3
dem frame 0
Dautien [[26.5   2.25  0.    0.   28.5   1.5  26.5   7.5  21.5   5.25 29.25 14.25
  31.25 22.5  34.5  29.5  24.25  2.25 24.25  8.5  23.25 11.25 25.5  14.25
  24.5  22.25 25.5  28.25  0.    0.    0.    0.    0.    0.    0.    0.  ]]
SHAPE (1, 36)
dem frame 1
SHAPE (2, 36)
dem frame 2
SHAPE (3, 36)
dem frame 3
SHAPE (4, 36)
dem frame 4
SHAPE (5, 36)
dem frame 5
SHAPE (6, 36)
dem frame 6
SHAPE (7, 36)
dem frame 7
SHAPE (8, 36)
dem frame 8
SHAPE (9, 36)
dem frame 9
SHAPE (10, 36)
dem frame 10
SHAPE (11, 36)
dem frame 11
SHAPE (12, 36)
dem frame 12
SHAPE (13, 36)
dem frame 13
SHAPE (14, 36)
dem frame 14
SHAPE (15, 36)
dem frame 15
SHAPE (16, 36)
dem frame 16
SHAPE (17, 36)
dem frame 17
SHAPE (18, 36)
dem frame 18
SHAPE (19, 36)
dem frame 19
SHAPE (20, 36)
dem frame 20
SHAPE (21, 36)
dem frame 21
SHAPE (22, 36)
dem frame 22
SHAPE (23, 36)
dem frame 23
SHAPE (24, 36)
dem frame 24
SHAPE (25, 36)
dem frame 25
SHAPE (26, 36)
dem frame 26
SHAPE (27, 36)
dem f

dem frame 17
SHAPE (18, 36)
dem frame 18
SHAPE (19, 36)
dem frame 19
SHAPE (20, 36)
dem frame 20
SHAPE (21, 36)
dem frame 21
SHAPE (22, 36)
dem frame 22
SHAPE (23, 36)
dem frame 23
SHAPE (24, 36)
dem frame 24
SHAPE (25, 36)
dem frame 25
SHAPE (26, 36)
dem frame 26
SHAPE (27, 36)
dem frame 27
SHAPE (28, 36)
dem frame 28
SHAPE (29, 36)
dem frame 29
SHAPE (30, 36)
dem frame 30
SHAPE (31, 36)
dem frame 31
SHAPE (32, 36)
SHAPEPRO (1, 32, 36)
SHAPEPRO [[[22.5   2.5   0.   ...  0.    0.    0.  ]
  [22.5   2.5   0.   ...  0.    0.    0.  ]
  [25.25  2.5  25.5  ...  0.    0.    0.  ]
  ...
  [48.5   3.25 49.5  ...  1.25  0.    0.  ]
  [48.5   3.25 49.5  ...  1.25  0.    0.  ]
  [24.5   1.5   0.   ...  0.    0.    0.  ]]]
input tensor([[[22.5000,  2.5000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [22.5000,  2.5000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [25.2500,  2.5000, 25.5000,  ...,  0.0000,  0.0000,  0.0000],
         ...,
         [48.5000,  3.2500, 49.5000,  ...,  1

dem frame 2
SHAPE (3, 36)
dem frame 3
SHAPE (4, 36)
dem frame 4
SHAPE (5, 36)
dem frame 5
SHAPE (6, 36)
dem frame 6
SHAPE (7, 36)
dem frame 7
SHAPE (8, 36)
dem frame 8
SHAPE (9, 36)
dem frame 9
SHAPE (10, 36)
dem frame 10
SHAPE (11, 36)
dem frame 11
SHAPE (12, 36)
dem frame 12
SHAPE (13, 36)
dem frame 13
SHAPE (14, 36)
dem frame 14
SHAPE (15, 36)
dem frame 15
SHAPE (16, 36)
dem frame 16
SHAPE (17, 36)
dem frame 17
SHAPE (18, 36)
dem frame 18
SHAPE (19, 36)
dem frame 19
SHAPE (20, 36)
dem frame 20
SHAPE (21, 36)
dem frame 21
SHAPE (22, 36)
dem frame 22
SHAPE (23, 36)
dem frame 23
SHAPE (24, 36)
dem frame 24
SHAPE (25, 36)
dem frame 25
SHAPE (26, 36)
dem frame 26
SHAPE (27, 36)
dem frame 27
SHAPE (28, 36)
dem frame 28
SHAPE (29, 36)
dem frame 29
SHAPE (30, 36)
dem frame 30
SHAPE (31, 36)
dem frame 31
SHAPE (32, 36)
SHAPEPRO (1, 32, 36)
SHAPEPRO [[[16.5   8.5  18.25 ...  1.25  0.    0.  ]
  [16.5   8.5  18.25 ...  1.25  0.    0.  ]
  [16.5   9.25 18.5  ...  1.25  0.    0.  ]
  ...
  [17.2

...

dem frame 27
SHAPE (28, 36)
dem frame 28
SHAPE (29, 36)
dem frame 29
SHAPE (30, 36)
dem frame 30
SHAPE (31, 36)
dem frame 31
SHAPE (32, 36)
SHAPEPRO (1, 32, 36)
SHAPEPRO [[[25.25  3.25  0.   ...  0.    0.    0.  ]
  [25.25  3.25  0.   ...  0.    0.    0.  ]
  [24.5   3.25  0.   ...  0.    0.    0.  ]
  ...
  [24.25  3.25  0.   ...  0.    0.    0.  ]
  [23.5   2.5   0.   ...  0.    0.    0.  ]
  [24.25  2.5   0.   ...  0.    0.    0.  ]]]
input tensor([[[25.2500,  3.2500,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [25.2500,  3.2500,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [24.5000,  3.2500,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         ...,
         [24.2500,  3.2500,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [23.5000,  2.5000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [24.2500,  2.5000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]]])
Tong hop) WAVING_2HANDS 3
dem frame 0
Dautien [[24.25  2.5   0.    0.   27.5   2.5  29.25  7.5  29.25 12.5  27

dem frame 2
SHAPE (3, 36)
dem frame 3
SHAPE (4, 36)
dem frame 4
SHAPE (5, 36)
dem frame 5
SHAPE (6, 36)
dem frame 6
SHAPE (7, 36)
dem frame 7
SHAPE (8, 36)
dem frame 8
SHAPE (9, 36)
dem frame 9
SHAPE (10, 36)
dem frame 10
SHAPE (11, 36)
dem frame 11
SHAPE (12, 36)
dem frame 12
SHAPE (13, 36)
dem frame 13
SHAPE (14, 36)
dem frame 14
SHAPE (15, 36)
dem frame 15
SHAPE (16, 36)
dem frame 16
SHAPE (17, 36)
dem frame 17
SHAPE (18, 36)
dem frame 18
SHAPE (19, 36)
dem frame 19
SHAPE (20, 36)
dem frame 20
SHAPE (21, 36)
dem frame 21
SHAPE (22, 36)
dem frame 22
SHAPE (23, 36)
dem frame 23
SHAPE (24, 36)
dem frame 24
SHAPE (25, 36)
dem frame 25
SHAPE (26, 36)
dem frame 26
SHAPE (27, 36)
dem frame 27
SHAPE (28, 36)
dem frame 28
SHAPE (29, 36)
dem frame 29
SHAPE (30, 36)
dem frame 30
SHAPE (31, 36)
dem frame 31
SHAPE (32, 36)
SHAPEPRO (1, 32, 36)
SHAPEPRO [[[18.5   0.    0.   ...  0.    0.    0.  ]
  [22.25  1.5   0.   ...  0.    0.    0.  ]
  [ 0.    0.    0.   ...  0.    0.    0.  ]
  ...
  [ 0. 

dem frame 27
SHAPE (28, 36)
dem frame 28
SHAPE (29, 36)
dem frame 29
SHAPE (30, 36)
dem frame 30
SHAPE (31, 36)
dem frame 31
SHAPE (32, 36)
SHAPEPRO (1, 32, 36)
SHAPEPRO [[[10.25  1.5   0.   ...  0.    0.    0.  ]
  [12.5   1.5   0.   ...  0.    0.    0.  ]
  [15.5   1.25  0.   ...  0.    0.    0.  ]
  ...
  [23.25  1.25  0.   ...  0.    0.    0.  ]
  [23.25  1.25  0.   ...  0.    0.    0.  ]
  [23.25  1.25  0.   ...  0.    0.    0.  ]]]
input tensor([[[10.2500,  1.5000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [12.5000,  1.5000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [15.5000,  1.2500,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         ...,
         [23.2500,  1.2500,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [23.2500,  1.2500,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [23.2500,  1.2500,  0.0000,  ...,  0.0000,  0.0000,  0.0000]]])
Tong hop) WAVING_2HANDS 3
dem frame 0
Dautien [[23.25  1.25  0.    0.   29.25  1.5  30.25  9.5  28.25 16.5  26

dem frame 0
Dautien [[42.5   3.5  43.25  2.5  43.5   3.5   0.    0.    0.    0.    0.    0.
   0.    0.    0.    0.   41.5   3.25  0.    0.    0.    0.    0.    0.
   0.    0.    0.    0.    0.    0.   43.5   2.5  42.5   2.5   0.    0.  ]]
SHAPE (1, 36)
dem frame 1
SHAPE (2, 36)
dem frame 2
SHAPE (3, 36)
dem frame 3
SHAPE (4, 36)
dem frame 4
SHAPE (5, 36)
dem frame 5
SHAPE (6, 36)
dem frame 6
SHAPE (7, 36)
dem frame 7
SHAPE (8, 36)
dem frame 8
SHAPE (9, 36)
dem frame 9
SHAPE (10, 36)
dem frame 10
SHAPE (11, 36)
dem frame 11
SHAPE (12, 36)
dem frame 12
SHAPE (13, 36)
dem frame 13
SHAPE (14, 36)
dem frame 14
SHAPE (15, 36)
dem frame 15
SHAPE (16, 36)
dem frame 16
SHAPE (17, 36)
dem frame 17
SHAPE (18, 36)
dem frame 18
SHAPE (19, 36)
dem frame 19
SHAPE (20, 36)
dem frame 20
SHAPE (21, 36)
dem frame 21
SHAPE (22, 36)
dem frame 22
SHAPE (23, 36)
dem frame 23
SHAPE (24, 36)
dem frame 24
SHAPE (25, 36)
dem frame 25
SHAPE (26, 36)
dem frame 26
SHAPE (27, 36)
dem frame 27
SHAPE (28, 36)
dem fra

dem frame 18
SHAPE (19, 36)
dem frame 19
SHAPE (20, 36)
dem frame 20
SHAPE (21, 36)
dem frame 21
SHAPE (22, 36)
dem frame 22
SHAPE (23, 36)
dem frame 23
SHAPE (24, 36)
dem frame 24
SHAPE (25, 36)
dem frame 25
SHAPE (26, 36)
dem frame 26
SHAPE (27, 36)
dem frame 27
SHAPE (28, 36)
dem frame 28
SHAPE (29, 36)
dem frame 29
SHAPE (30, 36)
dem frame 30
SHAPE (31, 36)
dem frame 31
SHAPE (32, 36)
SHAPEPRO (1, 32, 36)
SHAPEPRO [[[21.5   1.5   0.   ...  0.    0.    0.  ]
  [17.25  1.25  0.   ...  0.    0.    0.  ]
  [13.5   1.25  0.   ...  0.    0.    0.  ]
  ...
  [ 8.25  2.5   0.   ...  0.    0.    0.  ]
  [10.5   3.25  0.   ...  0.    0.    0.  ]
  [13.5   7.25  0.   ...  0.    0.    0.  ]]]
input tensor([[[21.5000,  1.5000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [17.2500,  1.2500,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [13.5000,  1.2500,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         ...,
         [ 8.2500,  2.5000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
  

# Convert to TensorRT

In [ ]:
import torch
from torch2trt import torch2trt


In [ ]:
model = LSTM(n_joints,n_hidden,n_categories,n_layer)
model.load_state_dict(torch.load('lstm_6_bn.pkl'))

In [ ]:
rnn.cuda().eval()

In [ ]:
x = torch.ones((1,32,36)).half().cuda()

In [ ]:
rnn(x)

In [ ]:
import pdb
pdb.pm()

In [ ]:
class LSTM(nn.Module):
    
    def __init__(self):
        super(LSTM,self).__init__()
        self.fc = torch.nn.Linear(12,1)
        
    def forward(self,inputs):
        out = self.fc(inputs)
        return out

In [ ]:
a = LSTM()

In [ ]:
a.cuda().eval()
x = torch.ones((1,12)).cuda()
model_trt = torch2trt(a, [x])

In [ ]:
wget https://github.com/xieyulai/LSTM-for-Human-Activity-Recognition-using-2D-Pose_Pytorch/blob/master/lstm.ipynb